In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


Code for preprocessing the text and other cleaning

Setting up of the Parameters

##Extra Section that will come in handy later

# This is an attempt to include the LASER multi lingual embedding


In [0]:
!git clone https://github.com/ceshine/LASER.git

Cloning into 'LASER'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 591 (delta 11), reused 24 (delta 11), pack-reused 560
Receiving objects: 100% (591/591), 2.60 MiB | 2.02 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [0]:
%env LASER=/content/LASER
!echo $LASER

env: LASER=/content/LASER
/content/LASER


In [0]:
%cd LASER

/content/LASER


In [0]:
!bash install_models.sh

 - creating directory /content/LASER/models
 - bilstm.93langs.2018-12-26.pt
 - 93langs.fcodes
 - 93langs.fvocab


In [0]:
!bash install_external_tools.sh

Installing external tools
 - creating directory /content/LASER/tools-external/moses-tokenizer/tokenizer
 - download tokenizer/tokenizer.perl
 - download tokenizer/detokenizer.perl
 - download tokenizer/normalize-punctuation.perl
 - download tokenizer/remove-non-printing-char.perl
 - download tokenizer/deescape-special-chars.perl
 - download tokenizer/lowercase.perl
 - download tokenizer/basic-protected-patterns
 - creating directory /content/LASER/tools-external/moses-tokenizer/share/nonbreaking_prefixes
 - download share/nonbreaking_prefixes/nonbreaking_prefix.ca
 - download share/nonbreaking_prefixes/nonbreaking_prefix.cs
 - download share/nonbreaking_prefixes/nonbreaking_prefix.de
 - download share/nonbreaking_prefixes/nonbreaking_prefix.el
 - download share/nonbreaking_prefixes/nonbreaking_prefix.en
 - download share/nonbreaking_prefixes/nonbreaking_prefix.es
 - download share/nonbreaking_prefixes/nonbreaking_prefix.fi
 - download share/nonbreaking_prefixes/nonbreaking_prefix.fr
 -

In [0]:
%cd tools-external/fastBPE/
! g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/LASER/tools-external/fastBPE


In [0]:
%cd ../../tasks/similarity/
! ls

/content/LASER/tasks/similarity
README.md  wmt.sh


In [0]:
! apt install libopenblas-base libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 8 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (127 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ..

In [0]:
! pip install faiss

     |████████████████████████████████| 4.7MB 44.6MB/s 


In [0]:
 % cd ./tasks/similarity
! bash wmt.sh

[Errno 2] No such file or directory: './tasks/similarity'
/content/LASER/tasks/similarity
 - Download WMT data
LASER: similarity search

Processing:
 - loading encoder /content/LASER/models/bilstm.93langs.2018-12-26.pt
 - transfer encoder to GPU


In [0]:
# sample feasibility check
# % cd ../../source/

import os
import sys
LASER = os.environ['LASER']
# now include the extra files in the source
sys.path.append(LASER + '/source')
sys.path.append(LASER + '/source/lib')

In [0]:
from embed import SentenceEncoder, EncodeLoad, EncodeFile
from text_processing import Token, BPEfastApply
from indexing import IndexCreate, IndexSearchMultiple, IndexPrintConfusionMatrix

ModuleNotFoundError: ignored

## These are the steps to replicate results of the LSTM file

In [0]:
MODEL_PATH = '/content/LASER/models/'
sentence_encoder = SentenceEncoder(
    str(MODEL_PATH + "bilstm.93langs.2018-12-26.pt"),
    max_sentences=None,
    max_tokens=10000,
    cpu=False)

## Load the entire dataset in this section

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

In [0]:
# Replace filename at the end
input_data = pd.read_csv('/gdrive/My Drive/data_23000_to_46000.csv')

In [0]:
input_data.columns

In [0]:
input_data.head()

### We would use review.text as the input and use it to predict review.primaryCategories

In [0]:
input_data.Product_Category.describe()

In [0]:
input_data = input_data.rename(columns={'Product_Category':'label'})
input_data.head()

In [0]:
LASER = os.environ['LASER']
FASTBPE = LASER + '/tools-external/fastBPE/fast'
MOSES_BDIR = LASER + '/tools-external/moses-tokenizer/tokenizer/'
MOSES_TOKENIZER = MOSES_BDIR + 'tokenizer.perl -q -no-escape -threads 20 -l '
MOSES_LC = MOSES_BDIR + 'lowercase.perl'
NORM_PUNC = MOSES_BDIR + 'normalize-punctuation.perl -l '
DESCAPE = MOSES_BDIR + 'deescape-special-chars.perl'
REM_NON_PRINT_CHAR = MOSES_BDIR + 'remove-non-printing-char.perl'
MECAB = LASER + '/tools-external/mecab'

In [0]:
from subprocess import check_output
def sample_token(string_val, lang='en'):
    tok = check_output(
                REM_NON_PRINT_CHAR
                + '|' + NORM_PUNC + lang
                + '|' + DESCAPE
                + '|' + MOSES_TOKENIZER + lang
                + ('| python3 -m jieba -d ' if lang == 'zh' else '')
                + ('|' + MECAB + '/bin/mecab -O wakati -b 50000 ' if lang == 'ja' else ''),
                input=string_val,
                encoding='UTF-8',
                shell=True)
    return tok.strip()

### Clean the data as that might cause problems later

In [0]:
index = input_data.reviewText.isnull()==True
samp = index[index==True]
input_data = input_data.drop(samp.index)

In [0]:
input_data.reviewText.isnull().sum() #Value should be always zero

0

In [0]:
"""
  Do the above mentioned operation on all the entries in the column. So,
  
"""
input_data['tokenized_review'] = input_data.reviewText.apply(sample_token)

In [0]:
input_data.head()

,reviewText,label,tokenized_review
0,"This is a really nice pocket knife, hurry and ...","Sports,Outdoors","This is a really nice pocket knife , hurry and..."
1,This is one of my favorite Outkast CDs. But th...,Digital Music,This is one of my favorite Outkast CDs . But t...
2,We bought this for our baking needs but have n...,Grocery and Gourmet,We bought this for our baking needs but have n...
3,"Alas, pianist Thelonious Monk was unable to ac...",CDs and Vinyl,"Alas , pianist Thelonious Monk was unable to a..."
4,"I bought this for my Sigma DMC-15E, it allows ...",Musical Instruments,"I bought this for my Sigma DMC-15E , it allows..."


In [0]:
bpe_codes = './LASER/models/93langs.fcodes'
bpe_vocab = bpe_codes.replace('fcodes', 'fvocab')


def apply_bpe(sentence):
  sentence = sentence.replace('\\',"")
  sentence = sentence.replace('"','\\"')
  bpe = check_output(
      'echo -n "'+sentence +'" | '+ FASTBPE + ' applybpe_stream '+  bpe_codes +' '+bpe_vocab,
      encoding='UTF-8',
      shell=True)
  return bpe.strip()


#input_data['tokenized_review_bpe'] = input_data.reviewText.apply(bpe_sentences)


In [0]:
tok_sentence = '"love the magnet easel ... " slow motion me "great for moving to different areas ... wish it had some sort of non skid pad on bottom though ...'
apply_bpe(tok_sentence)

'"@@ love the magn@@ et ea@@ sel ... " slo@@ w motion me "@@ great for mo@@ ving to different areas ... wish it had some sort of non sk@@ id pad on bot@@ tom though ...'

In [0]:
input_data.head()

,reviewText,label,tokenized_review
0,"This is a really nice pocket knife, hurry and ...","Sports,Outdoors","This is a really nice pocket knife , hurry and..."
1,This is one of my favorite Outkast CDs. But th...,Digital Music,This is one of my favorite Outkast CDs . But t...
2,We bought this for our baking needs but have n...,Grocery and Gourmet,We bought this for our baking needs but have n...
3,"Alas, pianist Thelonious Monk was unable to ac...",CDs and Vinyl,"Alas , pianist Thelonious Monk was unable to a..."
4,"I bought this for my Sigma DMC-15E, it allows ...",Musical Instruments,"I bought this for my Sigma DMC-15E , it allows..."


In [0]:
input_data.to_csv('/gdrive/My Drive/data_00000_to_23000_tokenized.csv')

In [0]:
input_data['tokenized_review_bpe'] = input_data.tokenized_review.apply(apply_bpe)

In [0]:
input_data.to_csv('/gdrive/My Drive/data_00000_to_23000_tokenized.csv')